In [4]:
%%sh
http_proxy=http://10.245.5.249:8080
export http_proxy
https_proxy=https://10.245.5.249:8080
export https_proxy

pip install gcsfs


In [29]:
import pandas as pd
import numpy as np
import gcsfs

In [30]:
df_storms = pd.read_csv('gs://aes-datahub-0001-curated/Outage_Restoration/Historical_Data/Master_Dataset/OMS_IPL_OUTAGE_DATASET_V8.6.csv')

In [31]:
df_cluster = pd.read_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Test/weather_source_clusters.csv')

In [32]:
df_storms.head()

,Unnamed: 0,OUTAGE_ID,INCIDENT_ID,STRCTUR_NO,EVENT,CREATION_DATETIME,ENERGIZED_DATETIME,CIRCT_ID,DNI_EQUIP_TYPE,SUBST_ID,...,clusters,Hour_Sin,Hour_Cos,MAJ_OTG_ID,LAST_CREATION_DATETIME,FIRST_CREATION_DATETIME,LAPSE_STORM_START_CREATIONDATETIME,RANK_SUBSEQUENT_MAJ_OTG_ID,WIND_DIRECTION,SEASON
0,0,2001317842,2.000391e+09,648-B/72,STORM,2007-02-24 20:02:00,2007-02-24 21:45:00,1702.0,ASWITCH,170.0,...,Cluster3.0,-0.866025,0.5,2.007000e+10,2007-02-24 22:49:02,2007-02-24 20:02:00,0.0000,1.0,S-E-E,WINTER
1,1,2001317865,2.000391e+09,237--/42,STORM,2007-02-24 20:18:26,2007-02-24 21:50:00,1402.0,FUSE,140.0,...,Cluster3.0,-0.866025,0.5,2.007000e+10,2007-02-24 22:49:02,2007-02-24 20:02:00,16.4333,2.0,S-E-E,WINTER
2,2,2001317856,2.000391e+09,M15-U/168,STORM,2007-02-24 20:31:55,2007-02-25 00:23:42,7001.0,1TBOH,700.0,...,Cluster7.0,-0.866025,0.5,2.007000e+10,2007-02-24 22:49:02,2007-02-24 20:02:00,29.9167,3.0,S-E-E,WINTER
3,3,2001317899,2.000391e+09,495-A/106,STORM,2007-02-24 20:56:03,2007-02-25 00:30:00,1257.0,FUSE,125.0,...,Cluster3.0,-0.866025,0.5,2.007000e+10,2007-02-24 22:49:02,2007-02-24 20:02:00,54.0500,4.0,S-E-E,WINTER
4,4,2001317873,2.000391e+09,801QA/2,STORM,2007-02-24 20:59:10,2007-02-24 21:15:02,7056.0,FUSE,705.0,...,Cluster3.0,-0.866025,0.5,2.007000e+10,2007-02-24 22:49:02,2007-02-24 20:02:00,57.1667,5.0,S-E-E,WINTER


In [33]:
print(df_storms.shape)

(19673, 107)


In [34]:
print(df_storms.OUTAGE_ID.nunique())

19673


In [35]:
print(list(df_storms.columns))

['Unnamed: 0', 'OUTAGE_ID', 'INCIDENT_ID', 'STRCTUR_NO', 'EVENT', 'CREATION_DATETIME', 'ENERGIZED_DATETIME', 'CIRCT_ID', 'DNI_EQUIP_TYPE', 'SUBST_ID', 'CALL_QTY', 'DOWNSTREAM_CUST_QTY', 'KEY_CUST_QTY', 'ETR_DATETIME', 'CUST_QTY', 'DOWNSTREAM_KVA_VAL', 'KVA_VAL', 'DAY_FLAG', 'TTR', 'POLE_CLUE_FLG', 'PART_LIGHT_CLUE_FLG', 'EMERGENCY_CLUE_FLG', 'POWER_OUT_CLUE_FLG', 'OPEN_DEVICE_CLUE_FLG', 'TREE_CLUE_FLG', 'WIRE_DOWN_CLUE_FLG', 'IVR_CLUE_FLG', 'EQUIPMENT_CLUE_FLG', 'TRANSFORMER_CLUE_FLG', 'OH_CAUSE_FLG', 'UG_CAUSE_FLG', 'ANIMAL_CAUSE_FLG', 'WEATHER_CAUSE_FLG', 'WEATHER_COLD_CAUSE_FLG', 'PUBLIC_CAUSE_FLG', 'WEATHER_LIGHTNING_CAUSE_FLG', 'WEATHER__SNOW_CAUSE_FLG', 'WEATHER__WIND_CAUSE_FLG', 'WEATHER__HEAT_CAUSE_FLG', 'CUST_REQUEST_CAUSE_FLG', 'WEATHER__FLOOD_CAUSE_FLG', 'STREET_CAUSE_FLG', 'SUBSTATION_CAUSE_FLG', 'TREE_CAUSE_FLG', 'MISCELLANEOUS_CAUSE_FLG', 'NO_CAUSE_FLG', 'PLANNED_CAUSE_FLG', 'NO_OUTAGE_CAUSE_FLG', 'FUSE_OCCURN_FLG', 'CUST_EQUIP_OCCURN_FLG', 'POLE_OCCURN_FLG', 'TRANSFORMER

In [36]:
df_storms['CREATION_DATETIME'] = pd.to_datetime(df_storms['CREATION_DATETIME'], errors = 'coerce')
df_storms['Date'] = df_storms['CREATION_DATETIME'].dt.date

In [37]:
df_cluster_profiling = df_storms.groupby(['Date', 'Marker_Location'], as_index=False).agg({'CUST_QTY' : 'sum', 'KVA_VAL' : 'sum', 'TTR' : 'sum', 'OUTAGE_ID' : 'count'})
df_cluster_profiling.rename(columns = {'OUTAGE_ID' : 'NO_OF_OUTAGES'}, inplace=True)

In [38]:
sum(df_cluster_profiling.NO_OF_OUTAGES)

19673

In [39]:
df_cluster_profiling.head()

,Date,Marker_Location,CUST_QTY,KVA_VAL,TTR,NO_OF_OUTAGES
0,2007-02-24,Marker11,271.0,503.666667,213.9500,1
1,2007-02-24,Marker12,1291.0,6221.333333,171.1833,2
2,2007-02-24,Marker18,70.0,870.000000,15.8667,1
3,2007-02-24,Marker19,1.0,15.000000,231.7833,1
4,2007-02-24,Marker3,858.0,5645.033333,411.9166,4


In [41]:
df_cluster_profiling.Marker_Location.unique()

array(['Marker11', 'Marker12', 'Marker18', 'Marker19', 'Marker3',
       'Marker10', 'Marker16', 'Marker5', 'Marker9', 'Marker1',
       'Marker13', 'Marker14', 'Marker15', 'Marker2', 'Marker4',
       'Marker7', 'Marker8', 'Marker6', 'Marker17', 'Marker20',
       'MarkerNone'], dtype=object)

In [42]:
df_cluster.head()

,Unnamed: 0,cldCvrAvg,cldCvrMax,cldCvrMin,dewPtAvg,dewPtMax,dewPtMin,feelsLikeAvg,feelsLikeMax,feelsLikeMin,...,windSpd100mMin,windSpd80mAvg,windSpd80mMax,windSpd80mMin,windSpdAvg,windSpdMax,windSpdMin,Location,clusters,clustersdivisive
0,5100,71,100,4,24.0,34.2,16.7,16.4,26.5,5.9,...,16.3,23.6,34.0,16.5,20.4,30.4,13.4,Marker1,4,1
1,4675,65,100,2,24.3,34.5,17.3,17.8,27.8,7.0,...,16.1,23.4,33.8,16.3,20.4,30.7,13.3,Marker10,4,1
2,3400,65,100,2,24.8,35.1,17.8,18.3,28.3,7.6,...,16.0,23.4,33.8,16.3,20.3,30.7,13.3,Marker11,4,1
3,7225,64,100,3,24.6,34.9,17.5,18.0,27.4,7.3,...,15.7,23.1,33.6,15.9,20.0,30.4,13.1,Marker12,4,1
4,7650,63,100,4,24.6,34.9,17.4,17.5,26.2,6.6,...,15.4,22.7,33.0,15.6,19.8,29.9,12.9,Marker13,4,1


In [45]:
df_cluster['timestamp'] = pd.to_datetime(df_cluster['timestamp'], errors='coerce', utc=True)
df_cluster['Date'] = df_cluster['timestamp'].dt.date

In [48]:
merge_clsuter_profiling = pd.merge(df_cluster_profiling, df_cluster, left_on=['Date','Marker_Location'], right_on=['Date', 'Location'], how='left')
merge_clsuter_profiling

,Date,Marker_Location,CUST_QTY,KVA_VAL,TTR,NO_OF_OUTAGES,Unnamed: 0,cldCvrAvg,cldCvrMax,cldCvrMin,...,windSpd100mMin,windSpd80mAvg,windSpd80mMax,windSpd80mMin,windSpdAvg,windSpdMax,windSpdMin,Location,clusters,clustersdivisive
0,2007-02-24,Marker11,271.0,503.666667,213.9500,1,3400.0,65.0,100.0,2.0,...,16.0,23.4,33.8,16.3,20.3,30.7,13.3,Marker11,4.0,1.0
1,2007-02-24,Marker12,1291.0,6221.333333,171.1833,2,7225.0,64.0,100.0,3.0,...,15.7,23.1,33.6,15.9,20.0,30.4,13.1,Marker12,4.0,1.0
2,2007-02-24,Marker18,70.0,870.000000,15.8667,1,850.0,60.0,100.0,4.0,...,16.0,23.6,34.6,16.3,20.4,30.6,13.3,Marker18,4.0,1.0
3,2007-02-24,Marker19,1.0,15.000000,231.7833,1,1275.0,60.0,100.0,3.0,...,16.3,23.9,36.0,16.5,20.6,30.4,13.6,Marker19,4.0,1.0
4,2007-02-24,Marker3,858.0,5645.033333,411.9166,4,5950.0,69.0,100.0,6.0,...,15.8,22.9,33.0,16.0,19.9,29.9,13.1,Marker3,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4505,2019-06-24,Marker4,16.0,100.000000,166.7500,1,424.0,52.0,100.0,0.0,...,8.4,14.5,19.3,9.0,4.5,7.3,2.6,Marker4,3.0,2.0
4506,2019-06-24,Marker5,8.0,0.000000,290.1667,1,6799.0,52.0,100.0,0.0,...,8.1,14.7,20.8,8.7,4.0,6.0,2.0,Marker5,3.0,2.0
4507,2019-06-24,Marker6,2.0,75.000000,191.4500,2,5949.0,53.0,100.0,0.0,...,9.1,14.8,19.3,9.7,4.6,7.2,1.8,Marker6,3.0,2.0
4508,2019-06-24,Marker7,17.0,50.000000,47.9333,1,4674.0,53.0,100.0,0.0,...,8.6,14.4,19.2,9.2,4.1,6.1,2.4,Marker7,3.0,2.0


In [49]:
merge_clsuter_profiling = merge_clsuter_profiling.loc[:, ~merge_clsuter_profiling.columns.str.contains('^Unnamed')]

In [50]:
print(list(merge_clsuter_profiling.columns))

['Date', 'Marker_Location', 'CUST_QTY', 'KVA_VAL', 'TTR', 'NO_OF_OUTAGES', 'cldCvrAvg', 'cldCvrMax', 'cldCvrMin', 'dewPtAvg', 'dewPtMax', 'dewPtMin', 'feelsLikeAvg', 'feelsLikeMax', 'feelsLikeMin', 'heatIndexAvg', 'heatIndexMax', 'heatIndexMin', 'latitude', 'longitude', 'mslPresAvg', 'mslPresMax', 'mslPresMin', 'precip', 'presTendAvg', 'presTendMax', 'presTendMin', 'radSolarAvg', 'radSolarMax', 'radSolarMin', 'radSolarTot', 'relHumAvg', 'relHumMax', 'relHumMin', 'sfcPresAvg', 'sfcPresMax', 'sfcPresMin', 'snowDepth', 'snowfall', 'spcHumAvg', 'spcHumMax', 'spcHumMin', 'tempAvg', 'tempMax', 'tempMin', 'timestamp', 'wetBulbAvg', 'wetBulbMax', 'wetBulbMin', 'windChillAvg', 'windChillMax', 'windChillMin', 'windDir100mAvg', 'windDir80mAvg', 'windDirAvg', 'windSpd100mAvg', 'windSpd100mMax', 'windSpd100mMin', 'windSpd80mAvg', 'windSpd80mMax', 'windSpd80mMin', 'windSpdAvg', 'windSpdMax', 'windSpdMin', 'Location', 'clusters', 'clustersdivisive']


In [53]:
merge_clsuter_profiling.to_csv('gs://aes-datahub-0002-curated/Outage_Restoration/Test/PROFILING_OMS.csv')